In [1]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.corpora import WikiCorpus
import jieba
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams['axes.unicode_minus'] = False

from cython.parallel import prange
from io import open
import pandas as pd
import logging
from wordcloud import WordCloud
import numpy as np

C:\Users\Zhang Bo Chao\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
"*******Data pre-processing, cut words and generate word list***********"
space = ' '
i = 0
l = []
zhwiki_name = 'C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/zhwiki-20190720-pages-articles-multistream.xml.bz2'
f = open('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/zhwiki.txt', 'w',encoding='utf-8')
wiki = WikiCorpus(zhwiki_name, lemmatize=False, dictionary={}) #从xml文件中读出训练语料
for text in wiki.get_texts():
    for i in prange(len(text), nogil=True):
        temp_sentence = text[i]
        seg_list = list(jieba.cut(temp_sentence)) #分词
        for temp_term in seg_list:
            l.append(temp_term)
    f.write(space.join(l) + '\n')
    l = []
    i = i + 1

    if (i %200 == 0):
        print('Saved ' + str(i) + ' articles')
f.close()
 

In [ ]:
"*******Generate model**********"
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

wiki_news = open('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/zhwiki.txt', 'r',encoding='utf-8')
print('~~~开始生成词向量模型~~~~请耐心等待')
model = Word2Vec(LineSentence(wiki_news), sg=0,size=200, window=5, min_count=5, workers=6)
print('~~~等待是有回报的~~~~词向量模型已经生成')
model.save('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/model/zhwiki_news.word2vec')


In [ ]:
"*******Test the model***********"
model = Word2Vec.load('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/model/zhwiki_news.word2vec')
print(model.similarity('数学', '经济学'))

result = pd.Series(model.most_similar(u'腾讯')) #查找近义相关词
print(result)

result1 = pd.Series(model.most_similar(u'人工智能', topn = 100))
print(result1)

print(model.wv['人工智能']) #查看人工智能的词向量（单个词语的词向量）


In [ ]:
"*******Create word cloud***********"
font_path = 'simkai.ttf'
 
'''
获取一个圆形的mask
'''
def get_mask():
    x, y = np.ogrid[:1000, :1000]
    mask = (x - 500) ** 2 + (y - 500) ** 2 > 500 ** 2
    mask = 255 * mask.astype(int)
    return mask
 
'''
绘制词云
'''
def draw_word_cloud(word_cloud):
    wc = WordCloud(font_path = font_path,background_color="white", mask=get_mask())
    wc.generate_from_frequencies(word_cloud)
    # 隐藏x轴和y轴
    plt.axis("off")
    plt.imshow(wc, interpolation="bilinear")
    plt.show()
    plt.savefig('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/word_cloud.png')

def draw():
    #logging.basicConfig(format="%(asctime)s:%(levelname)s:%(message)s", level=logging.INFO)
    model = Word2Vec.load("C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/model/zhwiki_news.word2vec")
    # 输入一个词找出相似的前100个词,默认是10个
    one_corpus = ["人工智能"]
    result = model.most_similar(one_corpus[0], topn=100)
    # 将返回的结果转换为字典,便于绘制词云
    word_cloud = dict()
    for sim in result:
        # print(sim[0],":",sim[1])
        word_cloud[sim[0]] = sim[1]
    # 绘制词云
    draw_word_cloud(word_cloud)

draw()


In [ ]:
"********Plotting and save fig**********"
def tsne_plot_by_keyword(model, keyword):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    keywords = pd.Series(model.most_similar(keyword, topn = 100)) # return a pandas series, get the most similar 100 words to the keyword, plot only those words
    
    for word in keywords:
        tokens.append(model[word[0]])
        labels.append(word[0])
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig('C:/Users/Zhang Bo Chao/Desktop/AI集训营/NLP必修课/Lecture4/Assignment/graph_by_keyword.png')
    plt.show()
    
tsne_plot_by_keyword(model, keyword = '人工智能')